### Sudoku with RPC (Go)

<figure style="float:right;border-right:2em solid white" >
    <img style="width:130pt" src="img/Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg"/>
    <figcaption style="width:130pt;font-size:small">A Sudoku puzzle.<br>&nbsp;</figcaption>
    <img style="width:130pt" src="img/Sudoku_Puzzle_by_L2G-20050714_solution_standardized_layout.svg"/>
    <figcaption style="width:130pt;font-size:small">A solution to above puzzle.</figcaption>
    <figcaption style="width:130pt;font-size:x-small"><a href="https://commons.wikimedia.org/wiki/File:Sudoku_Puzzle_by_L2G-20050714_solution_standardized_layout.svg" style="font-size:x-small">Wikimedia source.</a> <a href="https://creativecommons.org/licenses/by-sa/3.0/deed.en" style="font-size:x-small">Licensed under Creative Commons Attribution-Share Alike 3.0 Unported</a>
    </figcaption>
</figure>

Sudoku is a single-player game played on a 9 × 9 grid that is divided into nine 3 × 3 grids. The goal is to place digits 1 to 9 such that each row, each column, and each 3 × 3 grid contains each of the digits 1 to 9 exactly once, given a partially populated grid. Here, a Sudoku server allows players to request a puzzle and submit a solution. Each puzzle comes with a session ID. A player submits a solution together with the session ID and the player's name. The server checks if the solution is correct and times the response in milliseconds. Players are ranked on the time elapsed between requesting a puzzle and submitting a solution. The server keeps track of a leaderboard. 

Your task is to implement the player as a RPC client. Players may view the leaderboard. Try to reach the top of the leaderboard!

A solution that generates correct submissions to the server within the time bound of JupyterHub receives full points. The fastest solvers will receive up to 4 bonus points. You may use concurrent goroutines to speed up your solution.

*Server Side.* Here is the implementation of the server as it runs on JupyterHub. It is here just for reference, you do not need to run it. The server sends a partially populated puzzle with `0` entries indicating the blank spots.

In [ ]:
%%writefile sudokuserver.go
package main

import ("encoding/json"; "fmt"; "math/rand"; "net"; "net/rpc"; "os"; "sort"; "time")

type PuzzleSession struct {
	StartTime time.Time
	PuzzleIndex int
}

type PuzzleRequest struct {
	SessionID int
	Puzzle [][]int
}

type SudokuPuzzles struct {
	Unsolved [][][]int `json:"unsolved"`
// ''' go formatting '''
}

type PlayerSession struct {
	TotalTime int64
	PuzzlesSolved int
}

type PuzzleSolution struct { // sent from player to server
	SessionID int
	Solution [][]int
	Player string
}

type LeaderboardEntry struct { // entry in leaderboard
	Player string
	TimeTaken int64
}

type SudokuServer struct { // server state
	Puzzles [][][]int
	Leaderboard []LeaderboardEntry
	ActiveSessions map[int]PuzzleSession
	player_sessions map[string]PlayerSession
}

// RequestPuzzle provides a new puzzle to the player
func (s *SudokuServer) RequestPuzzle(args *int, reply *PuzzleRequest) error {
	sessionID := rand.Intn(10000)
	puzzleIndex := rand.Intn(len(s.Puzzles))
	s.ActiveSessions[sessionID] = PuzzleSession{StartTime: time.Now(), PuzzleIndex: puzzleIndex}
	*reply = PuzzleRequest{SessionID: sessionID, Puzzle: s.Puzzles[puzzleIndex]}
	return nil
}

// SubmitSolution receives the solution from the player
func (s *SudokuServer) SubmitSolution(args *PuzzleSolution, reply *string) error {
	session, ok := s.ActiveSessions[args.SessionID]
	if !ok {
		*reply = "Session not found."
		return nil
	}
	if !isValidSudoku(args.Solution, s.Puzzles[session.PuzzleIndex]) {
		*reply = "Incorrect solution."
		return nil
	}
	timeTakenMillis := time.Since(session.StartTime).Milliseconds()
	playerSession := s.player_sessions[args.Player]
	playerSession.TotalTime += timeTakenMillis
	playerSession.PuzzlesSolved++
	s.player_sessions[args.Player] = playerSession
	if playerSession.PuzzlesSolved == 2 {
		s.updateLeaderboard(args.Player, playerSession.TotalTime)
	}
	*reply = fmt.Sprintf("Correct solution! Time taken: %d milliseconds", timeTakenMillis)
	return nil
}

// updateLeaderboard updates the leaderboard with the new entry
func (s *SudokuServer) updateLeaderboard(player string, timeTaken int64) {
	entry := LeaderboardEntry{Player: player, TimeTaken: timeTaken}
	s.Leaderboard = append(s.Leaderboard, entry)
	sort.Slice(s.Leaderboard, func(i, j int) bool {
		return s.Leaderboard[i].TimeTaken < s.Leaderboard[j].TimeTaken
	})
	if len(s.Leaderboard) > 100 {
		s.Leaderboard = s.Leaderboard[:100]
	}
}

// GetLeaderboard returns the current leaderboard
func (s *SudokuServer) GetLeaderboard(args *int, reply *[]LeaderboardEntry) error {
	*reply = s.Leaderboard
	return nil
}

func isValidSudoku(solution [][]int, puzzle [][]int) bool {
	// Check if each row, column, and 3x3 subgrid contains all numbers from 1 to 9
	for i := 0; i < 9; i++ {
		if !isValidRow(solution, i) || !isValidColumn(solution, i) || !isValidBox(solution, i) {
			return false
		}
	}
	// Check if the solution matches the non-zero values of the puzzle
	for i := 0; i < 9; i++ {
		for j := 0; j < 9; j++ {
			if puzzle[i][j] != 0 && puzzle[i][j] != solution[i][j] {
				return false
			}
		}
	}
	return true
}

func isValidRow(board [][]int, row int) bool {
	seen := make(map[int]bool)
	for i := 0; i < 9; i++ {
		num := board[row][i]
		if num != 0 {
			if seen[num] {
				return false
			}
			seen[num] = true
		}
	}
	return true
}

func isValidColumn(board [][]int, col int) bool {
	seen := make(map[int]bool)
	for i := 0; i < 9; i++ {
		num := board[i][col]
		if num != 0 {
			if seen[num] {
				return false
			}
			seen[num] = true
		}
	}
	return true
}

func isValidBox(board [][]int, box int) bool {
	seen := make(map[int]bool)
	startRow := (box / 3) * 3
	startCol := (box % 3) * 3
	for i := 0; i < 3; i++ {
		for j := 0; j < 3; j++ {
			num := board[startRow + i][startCol + j]
			if num != 0 {
				if seen[num] {
					return false
				}
				seen[num] = true
			}
		}
	}
	return true
}

func loadPuzzles(filename string) [][][]int {
	file, err := os.Open(filename)
	if err != nil {panic(err)}
	defer file.Close()

	var puzzles SudokuPuzzles
	decoder := json.NewDecoder(file)
	err = decoder.Decode(&puzzles)
	if err != nil {panic(err)}

	return puzzles.Unsolved
}

func main() {
	rand.Seed(time.Now().UnixNano())

	puzzles := loadPuzzles("sudoku.json")
	if puzzles == nil {panic("Failed to load puzzles")}

	srv := &SudokuServer{
		Puzzles: puzzles,
		ActiveSessions: make(map[int]PuzzleSession),
		player_sessions: make(map[string]PlayerSession),
	}
	rpc.Register(srv)

	listener, err := net.Listen("tcp", ":8004")
	if err != nil {panic(err)}

	fmt.Println("Sudoku Server running on port 8004")
	for {
		conn, err := listener.Accept()
		if err != nil {
			fmt.Println("Accept error:", err)
			continue
		}
		go rpc.ServeConn(conn)
	}
}


*Client-Side.* Here is the template for the player with three places to complete:
- The first is for any auxiliary functions you may want to declare
- The second is for computing the solution. It should modify `puzzleResp.Puzzle`; you may print your solution.
- The third is for your nickname, under which you will be visible to everyone on the leaderboard. You may use your student ID, your name, or a pseudonym, but do not use your student number.

In [ ]:
%%writefile sudokuclient.go
package main

import ("fmt"; "net/rpc" )

type PuzzleRequest struct {
	SessionID int
	Puzzle [][]int
}

type PuzzleSolution struct {
	SessionID int
	Solution [][]int
	Player string
}

type LeaderboardEntry struct {
	Player string
	TimeTaken int64
}

func printPuzzle(puzzle [][]int) {
	for _, row := range puzzle {
		for _, val := range row {
			fmt.Printf("%d ", val)
		}
		fmt.Println()
	}
}

// auxiliary functions, if needed
# YOUR CODE HERE
raise NotImplementedError()

func main() {
	player, err := rpc.Dial("tcp", "localhost:8004")
	if err != nil {panic(err)}

	for i := 0; i < 2; i++ {
		var req int
		var puzzleResp PuzzleRequest
		err = player.Call("SudokuServer.RequestPuzzle", &req, &puzzleResp)
		if err != nil {panic(err)}

		fmt.Println("Puzzle received:")
		printPuzzle(puzzleResp.Puzzle)

		# YOUR CODE HERE
		raise NotImplementedError()
		
		var submitResp string
		solutionReq := PuzzleSolution{
			SessionID: puzzleResp.SessionID,
			Solution:  puzzleResp.Puzzle,
			# YOUR CODE HERE
			raise NotImplementedError()
		}
		err = player.Call("SudokuServer.SubmitSolution", &solutionReq, &submitResp)
		if err != nil {panic(err) }
		fmt.Println("Submit response:", submitResp)
	}

	// Viewing the leaderboard
	var leaderboard []LeaderboardEntry
	var req int
	err = player.Call("SudokuServer.GetLeaderboard", &req, &leaderboard)
	if err != nil {panic(err)}

	fmt.Println("Leaderboard:")
	for i, entry := range leaderboard {
		fmt.Printf("%d: %s - %.6d ms\n", i + 1, entry.Player, entry.TimeTaken)
	}
}

In [ ]:
!go run sudokuclient.go